In [2]:
pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.8 MB/s eta 0:00:00


In [16]:
import pandas as pd
import openai

openai.api_key = ""#api key 설정

df = pd.read_csv("lecture_comment.csv", encoding = "cp949")#csv파일 읽어오기, 한글 인코딩
df.dropna(inplace = True)#결측치 제외
df[['query', 'keyword']].rename(columns = {'query':'prompt', 'keyword' : 'completion'}).to_json("lecture_comment.jsonl",
                                            orient = 'records', lines = True, force_ascii = False)#json파일에 맞게 조정 후 json파일로 변형
create_file = openai.File.create(
      file = open("lecture_comment.jsonl"),
      purpose = 'fine-tune'
  )


In [12]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size = 0.2, random_state = 42)#훈련, 테스트 데이터셋 분리
len(train_df),len(test_df)

(4, 2)

In [22]:
import os
os.environ['OPENAI_API_KEY'] = ""#api key 인식 오류로 인한 설정

#마찬가지로 json 파일로 변형
train_df[['query', 'keyword']].rename(columns = {'query':'prompt', 'keyword' : 'completion'}).to_json("lecture_comment_train.jsonl", orient = 'records', lines = True, force_ascii = False)
test_df[['query', 'keyword']].rename(columns = {'query':'prompt', 'keyword' : 'completion'}).to_json("lecture_comment_test.jsonl", orient = 'records', lines = True, force_ascii = False)


In [38]:
!openai api fine_tunes.create -t lecture_comment_train.jsonl -m davinci#davinci 모델을 fine-tuning

Found potentially duplicated files with name 'lecture_comment_train.jsonl', purpose 'fine-tune' and size 2182 bytes
file-spsdvUoPhFYolEKy9nC3Kl89
Enter file ID to reuse an already uploaded file, or an empty string to upload this file anyway: lecture_comment_train_2.jsonl
File id 'lecture_comment_train_2.jsonl' is not among the IDs of the potentially duplicated files

Enter file ID to reuse an already uploaded file, or an empty string to upload this file anyway: lecture_comment_train.jsonl
File id 'lecture_comment_train.jsonl' is not among the IDs of the potentially duplicated files

Enter file ID to reuse an already uploaded file, or an empty string to upload this file anyway:  
Upload progress: 100% 2.18k/2.18k [00:00<00:00, 2.14Mit/s]
Uploaded file from lecture_comment_train.jsonl: file-IxXWQCzye4MEKU9GMAZKiktA
Created fine-tune: ft-XKNhzKC7bodH9ICgbnv0IMix
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-06-15 

In [41]:
!openai --api-key {""} api fine_tunes.follow -i ft-XKNhzKC7bodH9ICgbnv0IMix#fine-tuning한 모델로 학습 진행

[2023-06-15 01:12:20] Created fine-tune: ft-XKNhzKC7bodH9ICgbnv0IMix
[2023-06-15 01:13:19] Fine-tune costs $0.22
[2023-06-15 01:13:19] Fine-tune enqueued. Queue number: 0
[2023-06-15 01:23:25] Fine-tune started
[2023-06-15 01:26:11] Completed epoch 1/4
[2023-06-15 01:26:13] Completed epoch 2/4
[2023-06-15 01:26:16] Completed epoch 3/4
[2023-06-15 01:26:18] Completed epoch 4/4
[2023-06-15 01:26:50] Uploaded model: davinci:ft-personal-2023-06-15-01-26-50
[2023-06-15 01:26:51] Uploaded result file: file-BifFBggYBcxkQj6Lvh99n2e4
[2023-06-15 01:26:52] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m davinci:ft-personal-2023-06-15-01-26-50 -p <YOUR_PROMPT>


In [43]:
ft_ans = "davinci:ft-personal-2023-06-15-01-26-50"#fine-tuning 완료한 모델로 설정

def apply_ft_summarize(question, answering_model):
  prompt = f"다음 문장을 요약해줘 {question}\n",
  result = openai.Completion.create(model = answering_model, prompt = prompt, max_tokens = 300, temperature = 0, top_p = 1)#결과 생성
  return result['choices'][0]['text']
apply_ft_summarize("인강을 듣다보면 재미있는 선생님들이 많은것같아 더 재미있고 쉽게 수업을 들을 수 있는것 같다. 어떤 과학 선생님은 과도하게 오버해서 표정으로 수업을 하셨는데 재밌긴 재밌었는데 하고나면 왠지 그 선생님 표정과 그 수업내용이 쉽게 기억이 났다. 그리고 암기과목은 그냥 단순히 외울때는 힘이들었는데, 선생님들 특유의 잘 기억하는 법과 암기하는법을 말해줘서 시험때 많이 써먹었다. 그리고 어떤 선생님은 수업중에 자신의 친구를 얘기해주면서 실제 학교수업에서 잠깐 남은 자투리시간에 선생님들이 재밌는 얘기해주는것 같았다."
, ft_ans)

'\n제가 수업을 듣는데 재미있게 들을 수 있는 방법은 선생님들이 재밌는 얘기를 하는것이다. 어떤 선생님은 수업중에 자신의 친구를 얘기해주면서 실제 학교수업에서 잠깐 남은 자투리시간에 선생님들이 재밌는 얘기해주는것 같았다. 제가 수업을 듣는데 재미있'

In [46]:
import numpy as np


query_list = list()#질문 리스트
query_list = ["강의가 저한테 별로 맞지않아서 사실 지금은 강의를 듣지 않는 상황이라서...음.. 굳이 그래도 좋았던것은.. 아무래도 학교 수업이나 그런것은 너무 소위 잘하는 애들(잘하는 애들과 못하는 애들이 격차가 좀 있다고할가요..?)위주로 좀 정신없이 빠르게 지나가는 편인데 인터넷 강의는 그런게 없이 중간에 잠시 정지를 할 수 있을 뿐더러.. 아무래도 강의 선생님께서 경험같은것도 많고 하시다보니 어려운 공식 같은것도 쏙쏙 잘 들어오더라구요..그런점은 정말 좋았던것 같습니다~~  를 요약해줘", "일단..ㅋㅋㅋㅋㅋㅋㅋ그분 저희학교 한문쌤이랑 닮으셔서 집중이 캐잘돼요.ㅋㅋㅋㅋ 것보다도 자세하게 알려주셔서 감사한데 인생강의까지 중간중간에 넣어주시면서 알려주시는데 굉장히 재밌어요! 그치만 전 2.0으로 듣곤 하져.. 시간이 없으니까여^^;;; 근데 정말 구구절절 맞는 말씀만 해주셔서 2.0^^;;으로 해놓고도 귀에 쏙쏙 들어온답니다! ㅎㅎ. 엄청 자세히 알려주셔서 모르는 부분은 반복해서 듣다보면 어느새 전 다함ㅋ 우왕ㅋ 굳ㅋ 또 국어쌤! 완자 국어쌤 모세은선생님 그분은 화장이 진하셔서 기억에 남아요!ㅋㅋㅋ 슴옥히를 하셨는데 속쌍이라 그러신지 다 먹더라구요.. 제가봐도 안습ㅋㅋㅋㅋㅋㅋㅋㅋㅋ 또 수박씨 말고도 이비에스였나? 거기서 수학 남자선생님이ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 유진-차차를 합차나아지길 바래 오워~ 막 이러는거 아시는 분 풋쳐핸섬ㅋ대박웃겨요  에서 key phrase를 추출해줘"]

def return_ans(query):
  return query
def get_answer(query):
  try:
    response = openai.Completion.create(
        engine = "davinci", #모델 설정
        prompt = f"{query}\n를 요약하고 keyword를 추출해줘\n\n", #명령 설정
        temperature = 0, #다양성을 제어하는 변수, 0에 가까울수록 가장 확실한 답변을 생성한다.
        max_tokens = 257, #출력의 최대 길이를 제어하는 변수
        top_p = 1, #토큰의 상대적 중요도를 설정하는 변수, 1로 설정하면 모든 토큰을 고려한다.
        frequency_penalty = 0, #중복된 단어의 사용을 억제하는 변수, 값이 높을수록 반복되는 단어를 선택하지 않는다. 0이면 제약이 없다.
        presence_penalty = 0 #샘플링 결과에서 특정 토큰을 사용할 가능성을 제어하는 변수, 값이 높을수록 해당 토큰의 사용을 억제한다. 0이면 제약이 없다.
    )
    return response['choices'][0]['text']
  except Exception as e:
    print(e)
    return ""

answer_list = list()
for i in query_list:
  answer_list.append(get_answer(i))
print(answer_list)

["\n저는 이걸 받을 테니..\n(하, '㉜' )\n\n", '\n에 대해 아래 기태니일(?)\n- Keyword: "Key phrase" is a keyword that you want to rank for.\n\n - Extracting keywords from the article, and then extracting key phrases (keywords) by summarizing them in one sentence or two sentences are very important when writing an SEO optimized content because it will help your readers understand what they should know about this topic without having any confusion on how much information there would be inside of each paragraph/sentence within these paragraphs as well! It\'s also good practice if we can summarize our main points into 1~2 lines so people who read through those summaries could get their point across quickly instead reading all 5 pages worths\' amount text just like me..ㅎㅋ I\'m not saying my way works best but at least try doing something similar with yours too?']


In [10]:
print(len(answer_list))

2
